In [2]:
import os
import torch
from torch.utils.data import Dataset
from tqdm import tqdm
from airfrans.simulation import Simulation
import numpy as np
import json
from torch_geometric.data import Data

class AirfRANSGATDataset(Dataset):
    def __init__(self, root, task='scarce', train=True, k=10):
        self.root = root
        self.task = task
        self.train = train
        self.k = k

        taskk = 'full' if task == 'scarce' and not train else task
        split = 'train' if train else 'test'

        with open(os.path.join(root, 'manifest.json'), 'r') as f:
            manifest = json.load(f)[f"{taskk}_{split}"]

        self.names = manifest
        self.graphs = []

        for name in tqdm(manifest, desc=f'Loading AirfRANS ({taskk}, {split})'):
            sim = Simulation(root=root, name=name)

            inlet_velocity = (
                np.array([np.cos(sim.angle_of_attack), np.sin(sim.angle_of_attack)]) 
                * sim.inlet_velocity
            ).reshape(1, 2) * np.ones_like(sim.sdf)

            data = np.concatenate([
                sim.position,       
                inlet_velocity,     
                sim.sdf,           
                sim.normals,        
                sim.velocity,      
                sim.pressure,      
                sim.nu_t,           
            ], axis=-1)

            data = torch.tensor(data, dtype=torch.float32)
            x = data[:, :7]       
            y = data[:, 7:11]    

            #그래프 edge 생성
            edge_index = self._build_knn_graph(sim.position, k=self.k)

            self.graphs.append(Data(x=x, y=y, edge_index=edge_index))

    def __len__(self):
        return len(self.graphs)

    def __getitem__(self, idx):
        graph = self.graphs[idx]
        graph.name = self.names[idx]
        return graph

    def _build_knn_graph(self, pos, k=10):
        #(N, 2) numpy array를 edge_index tensor (2, E)로 반환
        from sklearn.neighbors import NearestNeighbors
        nbrs = NearestNeighbors(n_neighbors=k + 1).fit(pos)
        distances, indices = nbrs.kneighbors(pos)

        # i → j edges
        src, dst = [], []
        for i in range(len(indices)):
            for j in indices[i][1:]:  # skip self
                src.append(i)
                dst.append(j)
        edge_index = torch.tensor([src, dst], dtype=torch.long)
        return edge_index

from torch_geometric.loader import DataLoader

root = r"C:\airfran\Dataset"
dataset = AirfRANSGATDataset(root, task='scarce', train=True, k=10)
loader = DataLoader(dataset, batch_size=1, shuffle=True)

for batch in loader:
    print(batch.x.shape, batch.y.shape, batch.edge_index.shape)
    break


Loading AirfRANS (scarce, train): 100%|██████████| 200/200 [03:14<00:00,  1.03it/s]

torch.Size([169377, 7]) torch.Size([169377, 4]) torch.Size([2, 1693770])


In [11]:
import torch
from torch_geometric.nn import GATConv

class SimpleGAT(torch.nn.Module):
    def __init__(self, in_channels=7, hidden_channels=64, out_channels=4, heads=4):
        super().__init__()
        self.gat1 = GATConv(in_channels, hidden_channels, heads=heads, concat=True) #첫번째 레이어
        self.gat2 = GATConv(hidden_channels * heads, out_channels, heads=1, concat=False) #두번째 레이어

    def forward(self, x, edge_index):
        x = torch.relu(self.gat1(x, edge_index))
        x = self.gat2(x, edge_index)
        return x

# 사용 예시
model = SimpleGAT()
for batch in loader:
    pred = model(batch.x, batch.edge_index)
    print(pred.shape)  # (N, 4)
    break


torch.Size([180442, 4])
